In [1]:
from mogwai.core import MogwaiGraph
from mogwai.parser import graphml_to_mogwaigraph
from mogwai.core.traversal import MogwaiGraphTraversalSource
from mogwai.core.steps.statics import *

In [5]:
graph = graphml_to_mogwaigraph(
    "tests/documents/air-routes-latest.graphml",
    node_label_key="labelV",
    edge_label_key="labelE",
    node_name_key="code",
)
g = MogwaiGraphTraversalSource(graph)

In [6]:
g.V().element_map().range(1,2).to_list().run()

[{'labels': {'airport'},
  'name': 'airport',
  'labelV': 'airport',
  'type': 'airport',
  'code': 'ATL',
  'icao': 'KATL',
  'city': 'Atlanta',
  'desc': 'Hartsfield - Jackson Atlanta International Airport',
  'region': 'US-GA',
  'runways': 5,
  'longest': 12390,
  'elev': 1026,
  'country': 'US',
  'lat': 33.6366996765137,
  'lon': -84.4281005859375}]

In [5]:
g.V(1).outE().element_map().next().run()

{'labels': 'route', 'dist': 809, 'id': '3749'}

In [6]:
g.V().has('city', 'Maastricht').out().values('desc').to_list().run()

['Alicante International Airport',
 'Malta International Airport',
 'Istanbul International Airport',
 'Palma De Mallorca Airport',
 'Antalya International Airport',
 'Zakynthos, Dionysios Solomos Airport',
 'Faro Airport',
 'Francisco de Sa Carneiro Airport',
 'Kos Airport',
 'Tenerife South Airport',
 'Heraklion International Nikos Kazantzakis Airport',
 'Ioannis Kapodistrias International Airport',
 'Bari Karol Wojtyła Airport',
 'Catania-Fontanarossa Airport',
 'Girona Airport',
 'Vincenzo Florio Airport Trapani-Birgi',
 'Alghero-Fertilia Airport']

In [7]:
print(g.V().has('city','Frankfurt').union(out(), out().out()).order(desc=True).by("elev").values("city").dedup().limit(5).to_list().run())
print(g.V().has('city','Frankfurt').repeat(out()).times(2).emit().order(desc=True).by("elev").values("city").dedup().limit(5).to_list().run())

['Daocheng', 'Bangda', 'Kangding', 'La Paz / El Alto', 'Xigaze']
['Daocheng', 'Bangda', 'Kangding', 'La Paz / El Alto', 'Xigaze']


In [8]:
g.V().has("country", "US").repeat(outE().has("dist",lt(20)).inV()).times(5).simple_path().values("city").to_list().run()

['Nunapitchuk', 'Napaskiak']

In [10]:
g.V().has("country","DE").as_("start").union(
    has("runways",eq(1)).out().out(),
    has("runways",gte(1)).out()
).has('code','PMI').select("start").values("code").dedup().to_list().run()

[]

In [15]:
g.V().has("country","DE").element_map().next().run()

{'labels': {'airport'},
 'name': 'FRA',
 'type': 'airport',
 'icao': 'EDDF',
 'city': 'Frankfurt',
 'desc': 'Frankfurt am Main',
 'region': 'DE-HE',
 'runways': 4,
 'longest': 13123,
 'elev': 364,
 'country': 'DE',
 'lat': 50.0264015198,
 'lon': 8.54312992096}

In [4]:
g.V().has("country","DE").as_("start").union(
    has("runways",eq(1)).out().out(),
    has("runways",gte(1)).out()
).has('code','PMI').select("start").values("code").dedup().to_list().run()

['FRA',
 'MUC',
 'CGN',
 'HAM',
 'STR',
 'DUS',
 'HAJ',
 'DRS',
 'BRE',
 'ERF',
 'DTM',
 'NUE',
 'LEJ',
 'FMO',
 'FDH',
 'HHN',
 'FMM',
 'PAD',
 'NRN',
 'RLG',
 'FKB',
 'SCN',
 'LBC',
 'ZQW',
 'KSF',
 'XFW',
 'NDZ',
 'BER']

In [10]:
print(g.V().has("code", "DUS").out()\
	.or_(has('country', 'ES'),has('country', 'FR'))\
	.outE().as_("secondflight").inV().has("country","US")\
	.select("secondflight").outV().values("city").dedup().to_list().run())
print(g.V().has('code' , 'DUS').out()\
	.or_(has('country', 'ES'),has('country', 'FR')).as_("changeover")\
	.out().has('country','US').select("changeover").values('city').dedup().to_list().run())

['Paris', 'Barcelona', 'Madrid', 'Nice', 'Palma De Mallorca', 'Malaga', 'Tenerife Island']
['Paris', 'Barcelona', 'Madrid', 'Nice', 'Palma De Mallorca', 'Malaga', 'Tenerife Island']


In [2]:
graph = MogwaiGraph()
g = MogwaiGraphTraversalSource(graph)

In [3]:
jamie = g.addV("profile", "jamie").property("age", 50).next().run()
adam = g.addV("profile", "adam").property("age", 38).next().run()
g.addE("knows").from_(jamie).to_(adam).property("years", 15).next().run()

(0, 1)